In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [12]:
df = pd.read_csv('data/all_data.csv', parse_dates=['data_partida'])

In [13]:
# Sort por data
df = df.sort_values(by='data_partida')
df['temporada'] = df['data_partida'].dt.year

# Remover 'Q' do período para garantir ordem alfabética
df["periodo"] = df["periodo"].str.replace("Q", "", regex=False)


In [14]:
# Criar df com somente saldo das equipes
# Fica de update para melhorar ainda mais o modelo -> Colocar dados de Jogadores
teams_df = df[df['tipo'] == 'total']
teams_df = teams_df.sort_values(by=['data_partida', 'team', 'periodo'])

In [ ]:
# Criar um Hash para lidar o mesmo time de formas diferentes por temporada
teams_df['team_hash'] = teams_df['team'] # + teams_df['data_partida'].dt.year.astype(str)
teams_df['opp_hash'] = teams_df['oponente'] # + teams_df['data_partida'].dt.year.astype(str)

# Hash para o exato quarto da exata partida
teams_df['hash_tf'] = teams_df.apply(
    lambda row: ''.join(sorted([row['team_hash'], row['opp_hash']])) + str(row['periodo']),
    axis=1
)

# Hash para a partida
teams_df["hash_partida"] = teams_df.apply(
    lambda row: ''.join(sorted([row['team_hash'], row['opp_hash']])),
    axis=1
)




In [16]:
teams_df

,team,oponente,data_partida,periodo,tipo,nome,starter,min_sec,fg_made,fg_att,...,stl,blk,to,pf,site_tail,temporada,team_hash,opp_hash,hash_tf,hash_partida
40836,Bauru Basket,Pinheiros,2023-06-15,1,total,Total,0,0,14,19,...,4,1,6,4,9,2023,Bauru Basket2023,Pinheiros2023,Bauru Basket2023Pinheiros20231,Bauru Basket2023Pinheiros2023
40849,Bauru Basket,Pinheiros,2023-06-15,2,total,Total,0,0,13,20,...,3,0,6,4,7,2023,Bauru Basket2023,Pinheiros2023,Bauru Basket2023Pinheiros20232,Bauru Basket2023Pinheiros2023
40862,Bauru Basket,Pinheiros,2023-06-15,3,total,Total,0,0,8,14,...,1,0,3,7,0,2023,Bauru Basket2023,Pinheiros2023,Bauru Basket2023Pinheiros20233,Bauru Basket2023Pinheiros2023
40875,Bauru Basket,Pinheiros,2023-06-15,4,total,Total,0,0,15,21,...,5,0,3,3,11,2023,Bauru Basket2023,Pinheiros2023,Bauru Basket2023Pinheiros20234,Bauru Basket2023Pinheiros2023
40823,Bauru Basket,Pinheiros,2023-06-15,Final,total,Total,0,0,50,74,...,13,1,18,18,33,2023,Bauru Basket2023,Pinheiros2023,Bauru Basket2023Pinheiros2023Final,Bauru Basket2023Pinheiros2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13434,Vasco/Tijuca,B.Cearense,2025-08-04,1,total,Total,0,0,31,23,...,2,2,3,3,42,2025,Vasco/Tijuca2025,B.Cearense2025,B.Cearense2025Vasco/Tijuca20251,B.Cearense2025Vasco/Tijuca2025
13447,Vasco/Tijuca,B.Cearense,2025-08-04,2,total,Total,0,0,31,23,...,4,0,3,3,36,2025,Vasco/Tijuca2025,B.Cearense2025,B.Cearense2025Vasco/Tijuca20252,B.Cearense2025Vasco/Tijuca2025
13460,Vasco/Tijuca,B.Cearense,2025-08-04,3,total,Total,0,0,19,23,...,9,0,3,6,25,2025,Vasco/Tijuca2025,B.Cearense2025,B.Cearense2025Vasco/Tijuca20253,B.Cearense2025Vasco/Tijuca2025
13473,Vasco/Tijuca,B.Cearense,2025-08-04,4,total,Total,0,0,24,22,...,4,0,3,3,29,2025,Vasco/Tijuca2025,B.Cearense2025,B.Cearense2025Vasco/Tijuca20254,B.Cearense2025Vasco/Tijuca2025


In [17]:
# Colocar também o pace de cada equipe
teams_df['pace'] = teams_df['fg_att'] + teams_df['ft_att']*0.44 - teams_df['reb_of'] + teams_df['to']

In [18]:
mutual_stats = ['data_partida', 'periodo', 'temporada', 'hash_partida']
stats_to_keep = ['team_hash', 'fg_made', 'fg_att', 'fg_pct', 'fg3_made',
       'fg3_att', 'fg3_pct', 'fg2_made', 'fg2_att', 'fg2_pct', 'ft_made',
       'ft_att', 'ft_pct', 'reb_of', 'reb_def', 'reb_tot', 'ast', 'stl', 'blk',
       'to', 'pf', 'pace']

In [19]:
# Junta por hash_tf
df_unido = (
    teams_df
    .groupby('hash_tf')
    .apply(lambda g: pd.Series({
        # mutual (pega o primeiro, é o mesmo pros dois)
        **{col: g.iloc[0][col] for col in mutual_stats},

        # stats do time 1
        **{f"{col}_a": g.iloc[0][col] for col in stats_to_keep},

        # stats do time 2
        **{f"{col}_b": g.iloc[1][col] for col in stats_to_keep},
    }))
    .reset_index()
)


/tmp/ipykernel_441549/1945452029.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [20]:
df_unido

,hash_tf,data_partida,periodo,temporada,hash_partida,team_hash_a,fg_made_a,fg_att_a,fg_pct_a,fg3_made_a,...,ft_pct_b,reb_of_b,reb_def_b,reb_tot_b,ast_b,stl_b,blk_b,to_b,pf_b,pace_b
0,AABJ Joinville2023Caxias20231,2023-06-20,1,2023,AABJ Joinville2023Caxias2023,AABJ Joinville2023,23,17,52.9,4,...,50.0,7,4,11,3,0,2,3,5,15.76
1,AABJ Joinville2023Caxias20232,2023-06-20,2,2023,AABJ Joinville2023Caxias2023,AABJ Joinville2023,14,12,41.7,1,...,33.3,5,5,10,7,4,1,3,5,20.64
2,AABJ Joinville2023Caxias20233,2023-06-20,3,2023,AABJ Joinville2023Caxias2023,AABJ Joinville2023,9,14,28.6,0,...,66.7,1,8,9,5,4,1,5,6,19.96
3,AABJ Joinville2023Caxias20234,2023-06-20,4,2023,AABJ Joinville2023Caxias2023,AABJ Joinville2023,19,18,38.9,1,...,0.0,7,5,12,4,3,4,7,2,20.32
4,AABJ Joinville2023Caxias2023Final,2023-06-20,Final,2023,AABJ Joinville2023Caxias2023,AABJ Joinville2023,65,61,41.0,6,...,45.5,21,25,46,19,11,8,18,18,75.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2197,União Corinthians2024Vasco/Tijuca20241,2024-06-24,1,2024,União Corinthians2024Vasco/Tijuca2024,União Corinthians2024,19,16,43.8,2,...,66.7,3,7,10,6,3,0,4,5,20.96
2198,União Corinthians2024Vasco/Tijuca20242,2024-06-24,2,2024,União Corinthians2024Vasco/Tijuca2024,União Corinthians2024,9,14,21.4,1,...,60.0,1,10,11,7,4,1,3,4,19.20
2199,União Corinthians2024Vasco/Tijuca20243,2024-06-24,3,2024,União Corinthians2024Vasco/Tijuca2024,União Corinthians2024,12,12,41.7,0,...,37.5,3,6,9,6,2,0,7,6,22.52
2200,União Corinthians2024Vasco/Tijuca20244,2024-06-24,4,2024,União Corinthians2024Vasco/Tijuca2024,União Corinthians2024,14,17,29.4,3,...,66.7,2,9,11,4,1,1,4,6,17.64


In [21]:
df_unido['match_pace'] = np.mean(df_unido[['pace_a', 'pace_b']], axis=1)

In [22]:
cols_to_keep = [
    
    # Valores Categóricos
    'hash_partida',
    'hash_tf', 'data_partida',
    'periodo', 'temporada',
    'team_hash_a', 'team_hash_b',
    
    # =========== STATS ===========
    # TODO: Melhorar Modelo adicionando mais stats
    

    # Valores do Time A
    'fg_made_a', 'fg_att_a', 
    'fg3_made_a', 'fg3_att_a',
    'fg2_made_a', 'fg2_att_a',
    'ft_made_a', 'ft_att_a',

    # Valores do Time B
    'fg_made_b', 'fg_att_b',
    'fg3_made_b', 'fg3_att_b',
    'fg2_made_b', 'fg2_att_b',
    'ft_made_b', 'ft_att_b',

    # Valores de Pace
    'pace_a', 'pace_b', 'match_pace'

]

In [23]:
df = df_unido[cols_to_keep].copy()

In [24]:
# Colocar algumas colunas importantes de Pontuações
df['pts_a'] = df['fg3_made_a'] * 3 + df['fg2_made_a'] * 2 + df['ft_made_a']
df['pts_b'] = df['fg3_made_b'] * 3 + df['fg2_made_b'] * 2 + df['ft_made_b']
df['a_mov'] = df['pts_a'] - df['pts_b']
df['total'] = df['pts_a'] + df['pts_b']

In [28]:
df = df.sort_values(by=['data_partida', 'team_hash_b', 'periodo',])

In [26]:
# Agora, vamos precisar mapear a diferença de pontos no momento da partida
# Ex: 
    # Q1['a_mov'] == -12:
    # Q1['a_mov_prior'] == -12 

In [30]:
# Acumulado da diferença por partida
df["a_mov_cumsum"] = df.groupby("hash_partida")["a_mov"].cumsum()

# Diferença até o período anterior
df["a_mov_prior"] = df.groupby("hash_partida")["a_mov_cumsum"].shift(1).fillna(0)

# Se quiser, no Final você pode deixar NaN
df.loc[df["periodo"] == "Final", "a_mov_prior"] = np.nan

df

,hash_partida,hash_tf,data_partida,periodo,temporada,team_hash_a,team_hash_b,fg_made_a,fg_att_a,fg3_made_a,...,ft_att_b,pace_a,pace_b,match_pace,pts_a,pts_b,a_mov,total,a_mov_cumsum,a_mov_prior
574,Botafogo2023Mogi2023,Botafogo2023Mogi20231,2023-06-15,1,2023,Botafogo2023,Mogi2023,16,18,3,...,12,28.52,21.28,24.90,16,13,3,29,3,0.0
575,Botafogo2023Mogi2023,Botafogo2023Mogi20232,2023-06-15,2,2023,Botafogo2023,Mogi2023,20,10,2,...,11,22.28,20.84,21.56,20,16,4,36,7,3.0
576,Botafogo2023Mogi2023,Botafogo2023Mogi20233,2023-06-15,3,2023,Botafogo2023,Mogi2023,9,16,0,...,8,22.20,18.52,20.36,9,23,-14,32,-7,7.0
577,Botafogo2023Mogi2023,Botafogo2023Mogi20234,2023-06-15,4,2023,Botafogo2023,Mogi2023,11,19,0,...,13,22.76,17.72,20.24,11,17,-6,28,-13,-7.0
578,Botafogo2023Mogi2023,Botafogo2023Mogi2023Final,2023-06-15,Final,2023,Botafogo2023,Mogi2023,56,63,5,...,44,94.76,76.36,85.56,56,69,-13,125,-26,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,B.Cearense2025Vasco/Tijuca2025,B.Cearense2025Vasco/Tijuca20251,2025-08-04,1,2025,B.Cearense2025,Vasco/Tijuca2025,14,18,0,...,3,18.00,23.32,20.66,14,31,-17,45,-17,0.0
393,B.Cearense2025Vasco/Tijuca2025,B.Cearense2025Vasco/Tijuca20252,2025-08-04,2,2025,B.Cearense2025,Vasco/Tijuca2025,14,15,0,...,5,17.20,25.20,21.20,14,31,-17,45,-34,-17.0
394,B.Cearense2025Vasco/Tijuca2025,B.Cearense2025Vasco/Tijuca20253,2025-08-04,3,2025,B.Cearense2025,Vasco/Tijuca2025,15,15,0,...,7,19.32,23.08,21.20,15,19,-4,34,-38,-34.0
395,B.Cearense2025Vasco/Tijuca2025,B.Cearense2025Vasco/Tijuca20254,2025-08-04,4,2025,B.Cearense2025,Vasco/Tijuca2025,14,23,0,...,5,19.44,21.20,20.32,14,24,-10,38,-48,-38.0


In [36]:
mask = df['periodo'] == 'Final'

df_partida = df[mask]      # só os totais
df_quartos = df[~mask]     # todos os outros


In [37]:
df_partida

,hash_partida,hash_tf,data_partida,periodo,temporada,team_hash_a,team_hash_b,fg_made_a,fg_att_a,fg3_made_a,...,ft_att_b,pace_a,pace_b,match_pace,pts_a,pts_b,a_mov,total,a_mov_cumsum,a_mov_prior
578,Botafogo2023Mogi2023,Botafogo2023Mogi2023Final,2023-06-15,Final,2023,Botafogo2023,Mogi2023,56,63,5,...,44,94.76,76.36,85.56,56,69,-13,125,-26,NaN
431,Bauru Basket2023Pinheiros2023,Bauru Basket2023Pinheiros2023Final,2023-06-15,Final,2023,Bauru Basket2023,Pinheiros2023,50,74,3,...,23,85.16,79.12,82.14,50,93,-43,143,-86,NaN
1401,Flamengo2023São José Basketball2023,Flamengo2023São José Basketball2023Final,2023-06-15,Final,2023,Flamengo2023,São José Basketball2023,68,64,7,...,32,73.72,82.08,77.90,68,97,-29,165,-58,NaN
948,Caxias2023IVV/CETAF2023,Caxias2023IVV/CETAF2023Final,2023-06-16,Final,2023,Caxias2023,IVV/CETAF2023,74,70,7,...,18,77.16,61.92,69.54,74,40,34,114,68,NaN
1386,Flamengo2023Mogi2023,Flamengo2023Mogi2023Final,2023-06-16,Final,2023,Flamengo2023,Mogi2023,71,65,5,...,24,81.60,77.56,79.58,71,77,-6,148,-12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,Botafogo2025Flamengo2025,Botafogo2025Flamengo2025Final,2025-08-04,Final,2025,Botafogo2025,Flamengo2025,89,83,11,...,18,90.20,91.92,91.06,89,87,2,176,2,NaN
1080,Caxias2025Pato Basquete2025,Caxias2025Pato Basquete2025Final,2025-08-04,Final,2025,Caxias2025,Pato Basquete2025,94,75,12,...,13,78.60,65.72,72.16,94,72,22,166,44,NaN
1578,IVV/CETAF2025UNIFACISA2025,IVV/CETAF2025UNIFACISA2025Final,2025-08-04,Final,2025,IVV/CETAF2025,UNIFACISA2025,73,63,6,...,23,78.80,71.12,74.96,73,50,23,123,46,NaN
1714,Minas2025União Corinthians2025,Minas2025União Corinthians2025Final,2025-08-04,Final,2025,Minas2025,União Corinthians2025,112,74,15,...,21,84.36,66.24,75.30,112,44,68,156,136,NaN


In [ ]:
df.to_csv('all_data_integrated.csv', index=False)
df_partida.to_csv('matches_df_integrated.csv', index=False)
df_quartos.to_csv('quarters_df_integrated.csv', index=False)